In [ ]:
!pip install opencv-python
!pip install ultralytics

In [3]:
import cv2
import os
from datetime import datetime

def extract_frames_from_videos(source_folder, output_folder):
    """
    Belirtilen kaynak klasördeki tüm videolardan kareleri çıkarır ve belirtilen hedef klasöre kaydeder.

    :param source_folder: Video dosyalarının bulunduğu klasör
    :param output_folder: Çıktı karelerin kaydedileceği ana klasör
    """
    # Kaynak klasördeki tüm dosyaları al
    video_files = [f for f in os.listdir(source_folder) if f.endswith(('.mp4', '.avi', '.mov'))]

    if not video_files:
        print("Klasörde video bulunamadı.")
        return

    for video_file in video_files:
        video_path = os.path.join(source_folder, video_file)
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            print(f"{video_file} açılırken hata oluştu.")
            continue

        frame_count = 0
        video_name = os.path.splitext(video_file)[0]  # Dosya adını uzantısız al

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1

            # Klasör ismini belirleme
            folder_name = f"{video_name}_frame"

            video_output_directory = os.path.join(output_folder, folder_name)
            os.makedirs(video_output_directory, exist_ok=True)

            # Çıktı dosyasını oluştur
            output_file = os.path.join(video_output_directory, f"{video_name}_frame_{frame_count}.jpg")
            cv2.imwrite(output_file, frame)
        print(f"{video_file} - Frame ayrıldı")
        cap.release()

In [4]:
import os
import shutil
import random

def split_dataset(source_folder, output_folder, train_ratio=0.85):
    """
    Verilen klasördeki tüm dosyaları alır, belirtilen orana göre train ve val klasörlerine böler.

    :param source_folder: Veri kümesinin bulunduğu ana klasör
    :param output_folder: Çıktı dizini
    :param train_ratio: Eğitim verisi oranı (Varsayılan: 0.85)
    """
    train_images = os.path.join(output_folder, "train/images")
    train_labels = os.path.join(output_folder, "train/labels")
    val_images = os.path.join(output_folder, "val/images")
    val_labels = os.path.join(output_folder, "val/labels")

    os.makedirs(train_images, exist_ok=True)
    os.makedirs(train_labels, exist_ok=True)
    os.makedirs(val_images, exist_ok=True)
    os.makedirs(val_labels, exist_ok=True)

    image_files = []
    label_files = {}

    for root, _, files in os.walk(source_folder):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg')):
                full_path = os.path.join(root, file)
                image_files.append(full_path)
                label_files[full_path] = os.path.join(root, file.replace(file.split('.')[-1], 'txt'))

    random.shuffle(image_files)
    split_idx = int(len(image_files) * train_ratio)
    train_files = image_files[:split_idx]
    val_files = image_files[split_idx:]

    for img in train_files:
        shutil.move(img, os.path.join(train_images, os.path.basename(img)))
        label_path = label_files[img]
        if os.path.exists(label_path):
            shutil.move(label_path, os.path.join(train_labels, os.path.basename(label_path)))

    for img in val_files:
        shutil.move(img, os.path.join(val_images, os.path.basename(img)))
        label_path = label_files[img]
        if os.path.exists(label_path):
            shutil.move(label_path, os.path.join(val_labels, os.path.basename(label_path)))

    print(f"Veri kümesi bölme işlemi tamamlandı! Train: {len(train_files)}, Val: {len(val_files)}")


In [10]:
import os

def create_yaml_config(base_path, model_version, class_names):
    """
    YOLO eğitim için YAML konfigürasyon dosyasını oluşturur.

    :param base_path: Ana dizin yolu (örnek: "/content/drive/My Drive/colab/")
    :param model_version: Model sürüm numarası (örnek: "0.0.0")
    :param class_names: Algılanacak sınıf isimleri listesi (örnek: ["badem"])
    """
    model_path = os.path.join(base_path, f"model{model_version}")
    yaml_content = f"""train: {model_path}/train/images/
val: {model_path}/val/images/

nc: {len(class_names)}
names: {class_names}
"""

    yaml_path = os.path.join(model_path, "config.yaml")
    os.makedirs(model_path, exist_ok=True)

    with open(yaml_path, "w") as file:
        file.write(yaml_content)

    print(f"YAML dosyası oluşturuldu: {yaml_path}")
    
    
    
def main():
    base_path = os.getcwd()
    model_version = "0.0.0"
    class_names = ["badem"]
    
    create_yaml_config(base_path, model_version, class_names)

if __name__ is "__main__":
    main()

YAML dosyası oluşturuldu: /home/encoder/Desktop/ros2-robot/test/gomulu/model0.0.0/config.yaml


<>:36: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:36: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_5632/1821241668.py:36: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if __name__ is "__main__":


In [ ]:
import os
from ultralytics import YOLO

# Model ve eğitim parametreleri
ROOT_DIR = os.getcwd()
MODEL_PATH = "yolo11n.pt"  # Önceden eğitilmiş model
CONFIG_YAML = "config.yaml"
EPOCHS = 10  # Eğitim için epoch sayısı

# Eğitim ve doğrulama verilerini belirten YAML dosyası
data_path = os.path.join(ROOT_DIR, CONFIG_YAML)

# Modeli yükle
model = YOLO(MODEL_PATH)

# Modeli eğit
results = model.train(data=data_path, epochs=EPOCHS)

print("Model eğitimi tamamlandı!")

100%|██████████| 5.35M/5.35M [00:30<00:00, 182kB/s]

Model eğitimi tamamlandı!


'/home/encoder/Desktop/ros2-robot/test/gomulu'

In [ ]:
import cv2
from ultralytics import YOLO

def test_single_image(model_path, image_path, output_path):
    """
    YOLO modelini tek bir görüntü üzerinde test eder ve sonucu kaydeder.

    :param model_path: YOLO model dosyasının yolu (örnek: "trained_model/best.pt")
    :param image_path: Test edilecek görüntünün yolu
    :param output_path: Sonucun kaydedileceği klasör
    """
    model = YOLO(model_path)  # Eğitilmiş modeli yükle
    results = model(image_path)  # Görüntüyü modelde çalıştır

    # Görüntüyü oku
    img = cv2.imread(image_path)

    # Tespit edilen nesneleri ekrana çiz
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Koordinatları al
            conf = float(box.conf[0])  # Güven değeri
            label = result.names[int(box.cls[0])]  # Sınıf etiketi

            # Dikdörtgen çiz
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f"{label} {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Çıktıyı kaydet
    output_file = f"{output_path}/result.jpg"
    cv2.imwrite(output_file, img)

    print(f"Görüntü işleme tamamlandı, sonuç kaydedildi: {output_file}")

In [ ]:
import cv2

def test_video(model_path, video_path, output_path):
    """
    YOLO modelini bir video üzerinde test eder ve işlenmiş videoyu kaydeder.

    :param model_path: YOLO model dosyasının yolu (örnek: "trained_model/best.pt")
    :param video_path: Test edilecek video dosyasının yolu
    :param output_path: Sonucun kaydedileceği klasör
    """
    model = YOLO(model_path)  # Eğitilmiş modeli yükle
    cap = cv2.VideoCapture(video_path)  # Videoyu aç

    # Video özelliklerini al
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Video kaydedici ayarla
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 formatı
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_file = f"{output_path}/{video_name}_processed_video.mp4"
    out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Video bitti

        results = model(frame)  # Modeli kareye uygula

        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Koordinatları al
                conf = float(box.conf[0])  # Güven değeri
                label = result.names[int(box.cls[0])]  # Sınıf etiketi

                # Dikdörtgen çiz
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        out.write(frame)  # Çıkış videosuna kaydet

    cap.release()
    out.release()

    print(f"Video işleme tamamlandı, sonuç kaydedildi: {output_file}")

In [2]:
model_path = "/home/encoder/Desktop/ros2-robot/ros2_ws/src/tello_takip/tello_takip/best3.pt"

jpg_path = "/home/encoder/Downloads/video1_frame_43.jpg"

test_single_image(model_path,jpg_path,"/home/encoder/Desktop/ros2-robot/test")


image 1/1 /home/encoder/Downloads/video1_frame_43.jpg: 640x384 1 robot, 308.2ms
Speed: 6.4ms preprocess, 308.2ms inference, 14.7ms postprocess per image at shape (1, 3, 640, 384)
Görüntü işleme tamamlandı, sonuç kaydedildi: /home/encoder/Desktop/ros2-robot/test/result.jpg


In [ ]:
import cv2
import os

def process_video_frames(model_path, video_path, output_dir):
    """
    Bir videoyu karelere böler, YOLO modeli ile nesne tespiti yapar ve sonuçları kaydeder.

    :param model_path: YOLO model dosyasının yolu (örnek: "trained_model/best.pt")
    :param video_path: İşlenecek video dosyasının yolu
    :param output_dir: Çıkış klasörünün yolu (videoya özel klasör oluşturulacak)
    """
    # YOLO modelini yükle
    from ultralytics import YOLO
    model = YOLO(model_path)

    # Video adını al ve çıktı klasörlerini oluştur
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_output_folder = os.path.join(output_dir, video_name)
    images_folder = os.path.join(video_output_folder, "images")
    labels_folder = os.path.join(video_output_folder, "labels")

    os.makedirs(images_folder, exist_ok=True)
    os.makedirs(labels_folder, exist_ok=True)

    # Videoyu aç
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Video bittiğinde çık

        frame_count += 1
        results = model(frame)  # Modeli kareye uygula

        # Kareyi kaydet
        image_filename = f"frame_{frame_count:05d}.jpg"
        image_path = os.path.join(images_folder, image_filename)
        cv2.imwrite(image_path, frame)

        # Etiketleri kaydet
        label_filename = f"frame_{frame_count:05d}.txt"
        label_path = os.path.join(labels_folder, label_filename)

        with open(label_path, "w") as label_file:
            for result in results:
                for box in result.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])  # Nesnenin koordinatları
                    cls = int(box.cls[0])  # Sınıf etiketi
                    conf = float(box.conf[0])  # Güven skoru

                    # Normalleştirilmiş YOLO formatında etiket kaydet (x_center, y_center, width, height)
                    img_height, img_width = frame.shape[:2]
                    x_center = (x1 + x2) / 2 / img_width
                    y_center = (y1 + y2) / 2 / img_height
                    width = (x2 - x1) / img_width
                    height = (y2 - y1) / img_height

                    label_file.write(f"{cls} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {conf:.2f}\n")

    cap.release()
    print(f"Video işlendi: {video_output_folder}")